In [237]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np

### ● SUUMOの一つの物件から情報を取得する。

In [238]:
base_url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13109&cb=0.0&ct=9999999&mb=0&mt=9999999&et=9999999&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sngz=&po1=09"
response = requests.get(base_url)
soup     = BeautifulSoup(response.content, "lxml")
items    = soup.find(class_="cassetteitem")

In [239]:
# スクレイピングした HTML情報出力
f = open("test.txt", "w")
f.write(str(items))
f.close()

In [240]:
# 各物件情報の取得
property_name     = items.find(class_="cassetteitem_content-title").get_text() if items.find(class_="cassetteitem_content-title") else None
category          = items.find(class_="cassetteitem_content-label").span.get_text() if items.find(class_="cassetteitem_content-label") else None
address           = items.find(class_="cassetteitem_detail-col1").get_text() if items.find(class_="cassetteitem_detail-col1") else None
nearest_stations  = [station.get_text() for station in items.find_all(class_="cassetteitem_detail-text")] if items.find_all(class_="cassetteitem_detail-text") else None
construction_info = items.find(class_="cassetteitem_detail-col3").find_all("div") if items.find(class_="cassetteitem_detail-col3") else None
years_since_const = construction_info[0].get_text() if construction_info and len(construction_info) > 0 else None
number_of_floors  = construction_info[1].get_text() if construction_info and len(construction_info) > 1 else None
floor_number_td   = items.select_one("tr.js-cassette_link .cassetteitem_other-col03")
floor_number      = floor_number_td.get_text(strip=True) if floor_number_td else None
rent_info         = items.select_one(".cassetteitem_other tbody .js-cassette_link")
rent_admin_fee    = " / ".join([item.get_text(strip=True) for item in rent_info.select(".cassetteitem_price--rent, .cassetteitem_price--administration")]) if rent_info else None
deposit_gratuity  = " / ".join([price.get_text(strip=True) for price in rent_info.select(".cassetteitem_price--deposit, .cassetteitem_price--gratuity")]) if rent_info else None
layout_total_area = " / ".join([detail.get_text(strip=True) for detail in rent_info.select(".cassetteitem_madori, .cassetteitem_menseki")]) if rent_info else None


# 各物件情報の表示
print("物件名称 (Property Name):", property_name)
print("カテゴリー (Category):", category)
print("住所 (Address):", address)
print("最寄り駅 (Nearest Stations):", nearest_stations)
print("築年数 (Years Since Construction):", years_since_const)
print("階建 (Number of Floors):", number_of_floors)
print("階数 (Floor Number):", floor_number)
print("賃料/管理費 (Rent/Administration Fee):", rent_admin_fee)
print("敷金/礼金 (Deposit/Key Money):", deposit_gratuity)
print("間取り/占有面積 (Layout/Total Area):", layout_total_area)

物件名称 (Property Name): アーバン増渕
カテゴリー (Category): 賃貸マンション
住所 (Address): 東京都品川区西品川２
最寄り駅 (Nearest Stations): ['ＪＲ埼京線/大崎駅 歩10分', '東急大井町線/下神明駅 歩10分', 'ＪＲ京浜東北線/大井町駅 歩20分']
築年数 (Years Since Construction): 築38年
階建 (Number of Floors): 3階建
階数 (Floor Number): None
賃料/管理費 (Rent/Administration Fee): 15万円 / 5000円
敷金/礼金 (Deposit/Key Money): - / -
間取り/占有面積 (Layout/Total Area): 1LDK / 26.6m2


In [241]:
# 物件画像・間取り画像・詳細URLの取得
property_image_element = items.find(class_="cassetteitem_object-item")
property_image_url = property_image_element.img["rel"] if property_image_element and property_image_element.img else None

floor_plan_image_element = items.find(class_="casssetteitem_other-thumbnail")
floor_plan_image_url = floor_plan_image_element.img["rel"] if floor_plan_image_element and floor_plan_image_element.img else None

property_link_element = items.select_one("a[href*='/chintai/jnc_']")
property_link = "https://suumo.jp" + property_link_element['href'] if property_link_element else None

# 物件画像・間取り画像・詳細URLの表示
print("物件画像 URL (Property Image URL):", property_image_url)
print("間取り情報画像 URL (Floor Plan Image URL):", floor_plan_image_url)
print("物件リンク (Property Link):", property_link)

物件画像 URL (Property Image URL): https://img01.suumo.com/front/gazo/fr/bukken/743/100354045743/100354045743_gw.jpg
間取り情報画像 URL (Floor Plan Image URL): https://img01.suumo.com/front/gazo/fr/bukken/743/100354045743/100354045743_co.jpg
物件リンク (Property Link): https://suumo.jp/chintai/jnc_000086509817/?bc=100354045743


### ● SUUMOの複数ページから情報を取得する。

#####  検索範囲：東京都 中央区・港区・台東区・江戸川区・品川区・世田谷区
#####  検索条件：新着順
#####  表示数　：30件
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13102&sc=13103&sc=13106&sc=13123&sc=13109&sc=13112&cb=0.0&ct=9999999&mb=0&mt=9999999&et=9999999&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sngz=&po1=09

In [247]:
# 基本URLと最大ページ数の設定
base_url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13102&sc=13103&sc=13106&sc=13123&sc=13109&sc=13112&cb=0.0&ct=9999999&mb=0&mt=9999999&et=9999999&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sngz=&po1=09&page={}"
max_page = 100  # 最大ページ数

all_data = []

for page in range(1, max_page + 1):
    url = base_url.format(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')
    items = soup.findAll("div", {"class": "cassetteitem"})


    print("page", page, "items", len(items))

    for item in items:
        base_data = {}
        base_data["名称"]     = item.find("div", {"class": "cassetteitem_content-title"}).get_text(strip=True) if item.find("div", {"class": "cassetteitem_content-title"}) else None
        base_data["カテゴリ"] = item.find("div", {"class": "cassetteitem_content-label"}).span.get_text(strip=True) if item.find("div", {"class": "cassetteitem_content-label"}) else None
        base_data["アドレス"] = item.find("li", {"class": "cassetteitem_detail-col1"}).get_text(strip=True) if item.find("li", {"class": "cassetteitem_detail-col1"}) else None
        
        # 駅のアクセス情報をまとめて取得
        base_data["アクセス"] = ", ".join([station.get_text(strip=True) for station in item.findAll("div", {"class": "cassetteitem_detail-text"})])

        construction_info = item.find("li", {"class": "cassetteitem_detail-col3"}).find_all("div") if item.find("li", {"class": "cassetteitem_detail-col3"}) else None
        base_data["築年数"] = construction_info[0].get_text(strip=True) if construction_info and len(construction_info) > 0 else None
        base_data["構造"] = construction_info[1].get_text(strip=True) if construction_info and len(construction_info) > 1 else None

        tbodys = item.find("table", {"class": "cassetteitem_other"}).findAll("tbody")

        for tbody in tbodys:
            data = base_data.copy()
            # 階数情報の正確な取得
            floor_info = tbody.find_all("td")[2].get_text(strip=True) if len(tbody.find_all("td")) > 2 else None
            data["階数"]   = floor_info
            data["家賃"]   = tbody.select_one(".cassetteitem_price--rent").get_text(strip=True) if tbody.select_one(".cassetteitem_price--rent") else None
            data["管理費"] = tbody.select_one(".cassetteitem_price--administration").get_text(strip=True) if tbody.select_one(".cassetteitem_price--administration") else None
            data["敷金"]   = tbody.select_one(".cassetteitem_price--deposit").get_text(strip=True) if tbody.select_one(".cassetteitem_price--deposit") else None
            data["礼金"]   = tbody.select_one(".cassetteitem_price--gratuity").get_text(strip=True) if tbody.select_one(".cassetteitem_price--gratuity") else None
            data["間取り"] = tbody.select_one(".cassetteitem_madori").get_text(strip=True) if tbody.select_one(".cassetteitem_madori") else None
            data["面積"]   = tbody.select_one(".cassetteitem_menseki").get_text(strip=True) if tbody.select_one(".cassetteitem_menseki") else None

            # 物件画像・間取り画像・詳細URLの取得を最後に行う
            property_image_element = item.find(class_="cassetteitem_object-item")
            data["物件画像URL"] = property_image_element.img["rel"] if property_image_element and property_image_element.img else None

            floor_plan_image_element = item.find(class_="casssetteitem_other-thumbnail")
            data["間取画像URL"] = floor_plan_image_element.img["rel"] if floor_plan_image_element and floor_plan_image_element.img else None

            property_link_element = item.select_one("a[href*='/chintai/jnc_']")
            data["物件詳細URL"] = "https://suumo.jp" +property_link_element['href'] if property_link_element else None

            all_data.append(data)    


page 1 items 30
page 2 items 30
page 3 items 30
page 4 items 30
page 5 items 30
page 6 items 30
page 7 items 30
page 8 items 30
page 9 items 30
page 10 items 30
page 11 items 30
page 12 items 30
page 13 items 30
page 14 items 30
page 15 items 30
page 16 items 30
page 17 items 30
page 18 items 30
page 19 items 30
page 20 items 30
page 21 items 30
page 22 items 30
page 23 items 30
page 24 items 30
page 25 items 30
page 26 items 30
page 27 items 30
page 28 items 30
page 29 items 30
page 30 items 30
page 31 items 30
page 32 items 30
page 33 items 30
page 34 items 30
page 35 items 30
page 36 items 30
page 37 items 30
page 38 items 30
page 39 items 30
page 40 items 30
page 41 items 30
page 42 items 30
page 43 items 30
page 44 items 30
page 45 items 30
page 46 items 30
page 47 items 30
page 48 items 30
page 49 items 30
page 50 items 30
page 51 items 30
page 52 items 30
page 53 items 30
page 54 items 30
page 55 items 30
page 56 items 30
page 57 items 30
page 58 items 30
page 59 items 30
page 6

In [248]:
df = pd.DataFrame(all_data)
df = df.drop_duplicates() # 重複データの削除
df.head(2)

,名称,カテゴリ,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,礼金,間取り,面積,物件画像URL,間取画像URL,物件詳細URL
0,都営新宿線 篠崎駅 3階建 築13年,賃貸アパート,東京都江戸川区鹿骨６,"都営新宿線/篠崎駅 歩29分, ,",築13年,3階建,3階,3.9万円,2000円,-,-,ワンルーム,11.6m2,https://img01.suumo.com/front/gazo/fr/bukken/9...,https://img01.suumo.com/front/gazo/fr/bukken/9...,https://suumo.jp/chintai/jnc_000085867141/?bc=...
1,都営新宿線 篠崎駅 3階建 築13年,賃貸アパート,東京都江戸川区鹿骨６,"都営新宿線/篠崎駅 歩29分, ,",築13年,3階建,2階,3.7万円,2000円,-,-,ワンルーム,11.25m2,https://img01.suumo.com/front/gazo/fr/bukken/9...,https://img01.suumo.com/front/gazo/fr/bukken/9...,https://suumo.jp/chintai/jnc_000085867141/?bc=...


### ● Googleスプレッドシートへの書き込み・読み込み。

In [249]:
# google スプレッドシート 書き込み・読み込み
import gspread
from google.oauth2 import service_account
from google.oauth2.service_account import Credentials
from gspread_dataframe import get_as_dataframe
from gspread_dataframe import set_with_dataframe

In [250]:
# googleスプレッドシートの認証 jsonファイル読み込み(key値はGCPから取得)
SP_CREDENTIAL_FILE = "grspread_key.json"

scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

credentials = Credentials.from_service_account_file(
    SP_CREDENTIAL_FILE,
    scopes=scopes
)
gc = gspread.authorize(credentials)


SP_SHEET_KEY = '1oU1wQ59l8ID72PhOnazL639Jo3Dac2mPalKKaFiN3Gg' # d/〇〇/edit の〇〇部分
sh  = gc.open_by_key(SP_SHEET_KEY)

In [251]:
# 取得した不動産データの書き込み
SP_SHEET_wr     = 'tech0_00' # sheet名
worksheet_wr = sh.worksheet(SP_SHEET_wr) # シートのデータ取得
set_with_dataframe(worksheet_wr, df)

In [252]:
# 不動産データの取得
SP_SHEET     = 'tech0_00' # sheet名
worksheet = sh.worksheet(SP_SHEET) # シートのデータ取得
pre_data  = worksheet.get_all_values()
col_name = pre_data[0][:]
new_df = pd.DataFrame(pre_data[1:], columns=col_name) # 一段目をカラム、以下データフレームで取得

In [253]:
new_df.head(2)

,名称,カテゴリ,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,礼金,間取り,面積,物件画像URL,間取画像URL,物件詳細URL
0,都営新宿線 篠崎駅 3階建 築13年,賃貸アパート,東京都江戸川区鹿骨６,"都営新宿線/篠崎駅 歩29分, ,",築13年,3階建,3階,3.9万円,2000円,-,-,ワンルーム,11.6m2,https://img01.suumo.com/front/gazo/fr/bukken/9...,https://img01.suumo.com/front/gazo/fr/bukken/9...,https://suumo.jp/chintai/jnc_000085867141/?bc=...
1,都営新宿線 篠崎駅 3階建 築13年,賃貸アパート,東京都江戸川区鹿骨６,"都営新宿線/篠崎駅 歩29分, ,",築13年,3階建,2階,3.7万円,2000円,-,-,ワンルーム,11.25m2,https://img01.suumo.com/front/gazo/fr/bukken/9...,https://img01.suumo.com/front/gazo/fr/bukken/9...,https://suumo.jp/chintai/jnc_000085867141/?bc=...


### ● 不動産データを加工する。

In [254]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11743 entries, 0 to 11742
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   名称       11743 non-null  object
 1   カテゴリ     11743 non-null  object
 2   アドレス     11743 non-null  object
 3   アクセス     11743 non-null  object
 4   築年数      11743 non-null  object
 5   構造       11743 non-null  object
 6   階数       11743 non-null  object
 7   家賃       11743 non-null  object
 8   管理費      11743 non-null  object
 9   敷金       11743 non-null  object
 10  礼金       11743 non-null  object
 11  間取り      11743 non-null  object
 12  面積       11743 non-null  object
 13  物件画像URL  11743 non-null  object
 14  間取画像URL  11743 non-null  object
 15  物件詳細URL  11743 non-null  object
dtypes: object(16)
memory usage: 1.4+ MB


In [255]:
new_df['築年数'] = new_df["築年数"].apply( lambda x: 0 if x=='新築' else int(re.split('[築年]', x )[1]) )

In [256]:
def get_most_floor(x):
    if ('階建' not in x) :
        return np.nan
    elif('B' not in x) :
        list = re.findall(r'(\d+)階建',str(x))
        list = map(int, list)
        min_value = min(list)
        return min_value

new_df['構造'] = new_df['構造'].apply(get_most_floor)
print(new_df['構造'].head(5))

0    3
1    3
2    3
3    3
4    3
Name: 構造, dtype: int64


In [257]:
def get_floor(x):
    if ('階' not in x) :
        return np.nan
    elif('B' not in x) :
        list = re.findall(r'(\d+)階',str(x))
        # time_listを数値型に変換
        list = map(int, list)
        # time_listの最小値をmin_valueに代入
        min_value = min(list)
        return min_value
    else:
        list = re.findall(r'(\d+)階',str(x))
        # time_listを数値型に変換
        list = map(int, list)
        # time_listの最小値をmin_valueに代入
        min_value = -1*min(list)
        return min_value

new_df['階数'] = new_df['階数'].apply(get_floor)
print(new_df['階数'].head(5))

0    3.0
1    2.0
2    2.0
3    2.0
4    2.0
Name: 階数, dtype: float64


In [258]:
def change_fee(x):
    if ('万円' not in x) :
        return np.nan
    else:
        return float(x.split('万円')[0])

new_df['家賃'] = new_df['家賃'].apply(change_fee)
new_df['敷金'] = new_df['敷金'].apply(change_fee)
new_df['礼金'] = new_df['礼金'].apply(change_fee)

In [259]:
def change_fee2(x):
    if ('円' not in x) :
        return np.nan
    else:
        return float(x.split('円')[0])


new_df['管理費'] = new_df['管理費'].apply(change_fee2)

In [260]:
new_df['面積'] = new_df['面積'].apply(lambda x: float(x[:-2]))

In [261]:
new_df['区'] = new_df["アドレス"].apply(lambda x : x[x.find("都")+1:x.find("区")+1])

In [262]:
new_df['市町'] = new_df["アドレス"].apply(lambda x : x[x.find("区")+1 :-1])

In [266]:
def split_access(row):
    accesses = row['アクセス'].split(', ')
    results = {}

    for i, access in enumerate(accesses, start=1):
        if i > 3:
            break  # 最大3つのアクセス情報のみを考慮

        parts = access.split('/')
        if len(parts) == 2:
            line_station, walk = parts
            # ' 歩'で分割できるか確認
            if ' 歩' in walk:
                station, walk_min = walk.split(' 歩')
                # 歩数の分の数値だけを抽出
                walk_min = int(re.search(r'\d+', walk_min).group())
            else:
                station = None
                walk_min = None
        else:
            line_station = access
            station = walk_min = None

        results[f'アクセス①{i}線路名'] = line_station
        results[f'アクセス①{i}駅名'] = station
        results[f'アクセス①{i}徒歩(分)'] = walk_min

    return pd.Series(results)

# 新しい列をデータフレームに適用
new_df = new_df.join(new_df.apply(split_access, axis=1))

In [267]:
new_df.head(2)

,名称,カテゴリ,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,...,市町,アクセス①1線路名,アクセス①1駅名,アクセス①1徒歩(分),アクセス①2線路名,アクセス①2駅名,アクセス①2徒歩(分),アクセス①3線路名,アクセス①3駅名,アクセス①3徒歩(分)
0,都営新宿線 篠崎駅 3階建 築13年,賃貸アパート,東京都江戸川区鹿骨６,"都営新宿線/篠崎駅 歩29分, ,",13,3,3.0,3.9,2000.0,NaN,...,鹿骨,都営新宿線,篠崎駅,29,,None,NaN,,None,NaN
1,都営新宿線 篠崎駅 3階建 築13年,賃貸アパート,東京都江戸川区鹿骨６,"都営新宿線/篠崎駅 歩29分, ,",13,3,2.0,3.7,2000.0,NaN,...,鹿骨,都営新宿線,篠崎駅,29,,None,NaN,,None,NaN


In [268]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11743 entries, 0 to 11742
Data columns (total 27 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   名称           11743 non-null  object 
 1   カテゴリ         11743 non-null  object 
 2   アドレス         11743 non-null  object 
 3   アクセス         11743 non-null  object 
 4   築年数          11743 non-null  int64  
 5   構造           11743 non-null  int64  
 6   階数           11740 non-null  float64
 7   家賃           11743 non-null  float64
 8   管理費          10969 non-null  float64
 9   敷金           7589 non-null   float64
 10  礼金           5955 non-null   float64
 11  間取り          11743 non-null  object 
 12  面積           11743 non-null  float64
 13  物件画像URL      11743 non-null  object 
 14  間取画像URL      11743 non-null  object 
 15  物件詳細URL      11743 non-null  object 
 16  区            11743 non-null  object 
 17  市町           11743 non-null  object 
 18  アクセス①1線路名    11743 non-null  object 
 19  アクセス

In [269]:
# 取得した不動産データの書き込み
SP_SHEET_wr     = 'tech0_01' # sheet名
worksheet_wr = sh.worksheet(SP_SHEET_wr) # シートのデータ取得
set_with_dataframe(worksheet_wr, new_df)